In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://huggingface.co/cointegrated/LaBSE-en-ru
https://huggingface.co/DeepPavlov/rubert-base-cased

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install transformers datasets  accelerate

In [218]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import evaluate
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,BertTokenizer, BertForSequenceClassification,DataCollatorWithPadding,  AutoModel

# Взаимоотношение между текстами

In [ ]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
if torch.cuda.is_available():
    model.cuda()

In [ ]:
text1 = 'Сократ - человек, а все люди смертны.'
text2 = 'Сократ никогда не умрёт.'

In [ ]:
with torch.inference_mode():
    out = model(**tokenizer(text1, text2, return_tensors='pt').to(model.device))
    proba = torch.softmax(out.logits, -1).cpu().numpy()[0]

In [ ]:
print({v: proba[k] for k, v in model.config.id2label.items()})

# Классификация текстов

In [150]:
model_name = "cointegrated/rubert-tiny2"  # компактная русскоязычная модель
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # если у вас два класса

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [151]:
if torch.cuda.is_available():
    model.cuda()

In [172]:
train = pd.read_csv('/content/drive/MyDrive/vseros/train.csv',sep = '\t',index_col  = False)

In [173]:
train = train.drop(columns = ['idx'])

In [174]:
train

,Score,Text
0,Positive,В Альфа-Банке работает замечательная девушка -...
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,Positive,Очень порадовала оперативность работы в банке....
3,Negative,Имела неосторожность оформить потреб. кредит в...
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...
13997,Positive,Добрый день! Давно являюсь клиентом банка Русс...


In [175]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [176]:
train['Score'] = le.fit_transform(train['Score'])

In [177]:
train = Dataset.from_pandas(train)

In [178]:
def preprocess_function(examples):
    return tokenizer(examples["Text"],truncation=True,max_length = 512)

In [179]:
train

Dataset({
    features: ['Score', 'Text'],
    num_rows: 13999
})

In [180]:
len(set([elem['Score'] for elem in train]))

2

In [181]:
train[0]

{'Score': 1,
 'Text': 'В Альфа-Банке работает замечательная девушка - Ильясова Орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время моего прихода, помогла оформить кредит в размере 1млн рублей, когда я пришла с партнером (передавать ей полученный кредит за покупаемый мною авто), специалист Ильясова Орна помогла нам вывести всю сумму в один день (а это было непросто), так что сделка состоялась и все остались довольны! Моя знакомая в конце всего добавила: Теперь я \xa0поняла, почему мы пришли в это отделение, к этой девушке..Побольше бы таких замечательных специалистов! Приобретать программу здоровье, и вообще все, что связано с Альфа-Банком, теперь буду только у нее!'}

In [182]:
train = train.map(preprocess_function,remove_columns=['Text'])

Map:   0%|          | 0/13999 [00:00<?, ? examples/s]

In [183]:
train = train.rename_column("Score", "label")

In [184]:
train

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 13999
})

In [185]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [186]:
accuracy = evaluate.load("accuracy")

In [187]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [188]:
training_args = TrainingArguments(
    output_dir="cache",
    learning_rate=3e-4,
    per_device_train_batch_size=64,
    #per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    #load_best_model_at_end=True
)

# Создание тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    #eval_dataset=val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [189]:
# Обучение модели
trainer.train()

Step,Training Loss


TrainOutput(global_step=438, training_loss=0.13327499285136182, metrics={'train_runtime': 204.399, 'train_samples_per_second': 136.977, 'train_steps_per_second': 2.143, 'total_flos': 206463046914048.0, 'train_loss': 0.13327499285136182, 'epoch': 2.0})

In [193]:
torch.cuda.empty_cache()

In [194]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

In [195]:
del train
del val

NameError: name 'train' is not defined

In [196]:
from tqdm import tqdm

In [201]:
test = pd.read_csv('/content/drive/MyDrive/vseros/test.csv',sep = '\t',index_col  = 0)

In [202]:
test

,Text
idx,
13999,Замечательная история об открытии счета в банк...
14000,"23.07.14г. я досрочно внес сумму, указанную оп..."
14001,Была задолженность по кредитной карте. Банк по...
14002,Срочно понадобилась определенная денежная сумм...
14003,Здравствуйте. В местном отделении Бинбанка г. ...
...,...
19994,Уважаемые представители Русфинанс Банка! Прошу...
19995,04 июня сняли с дебетовой карты банка 10 000 T...
19996,В апреле решил я купить шуруповерт. Посмотрел ...


In [203]:
lst=[]
with torch.no_grad():
    for input in tqdm(test['Text']):
        t = tokenizer(input, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**{k: v.to(model.device) for k, v in t.items()}).logits.argmax(-1)
        lst.append(int(outputs.cpu()))

100%|██████████| 6000/6000 [00:52<00:00, 114.05it/s]


In [ ]:
#print(accuracy.compute(predictions=lst, references=label))

{'accuracy': 0.7193333333333334}


In [207]:
subm = pd.read_csv('/content/drive/MyDrive/vseros/sample_submission.csv',sep = '\t')

In [209]:
subm['Score'] = le.inverse_transform(lst)

In [210]:
subm

,idx,Score
0,13999,Negative
1,14000,Negative
2,14001,Negative
3,14002,Negative
4,14003,Negative
...,...,...
5995,19994,Positive
5996,19995,Positive
5997,19996,Negative
5998,19997,Positive


In [212]:
subm.to_csv('subm.csv',index = False,sep = '\t')

In [214]:
model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')

In [219]:
class CustomModel(nn.Module):
    def __init__(self, base_model):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        self.custom_layer = nn.Linear(base_model.config.hidden_size, 1)  # Выход 1 значение для регрессии
        self.sigmoid = nn.Sigmoid()  # Сигмоид для выхода от 0 до 1

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        logits = self.custom_layer(sequence_output[:, 0, :])  # Используем [CLS] токен
        output = self.sigmoid(logits)  # Применяем сигмоид для получения значения от 0 до 1
        return output

In [220]:
model = CustomModel(model)

In [221]:
model

CustomModel(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementw

In [222]:
lst=[]
with torch.no_grad():
    for input in tqdm(test['Text']):
        t = tokenizer(input, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**{k: v.to(model.device) for k, v in t.items()})
        predicted_value = outputs.cpu().numpy().flatten()[0]  # Извлекаем значение и преобразуем в numpy
        lst.append(float(predicted_value))

  0%|          | 0/6000 [00:00<?, ?it/s]


AttributeError: 'CustomModel' object has no attribute 'device'